In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('biv_clustered.csv', encoding='cp1251', sep=',', low_memory=False)
pd.set_option('display.max_columns', None)
df.drop('Unnamed: 0', axis= 1 , inplace= True )
df.head(5)

,uid,name,category1_name,category2_name,category3_name,category4_name,nn_clusters_relu
0,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,107
1,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,107
2,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,107
3,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,107
4,00e7d508-7577-11e9-80ca-901b0e95a2a8,Тент ENO ProFly XL Sil Lichen,Палатки,Тенты и противомоскитные пологи,NaN,NaN,107


In [3]:
# Группировка данных по кластерам
cluster_groups = df.groupby('nn_clusters_relu')['uid'].apply(list).reset_index()

In [4]:
cluster_groups

,nn_clusters_relu,uid
0,0,"[04cf8541-e440-11ed-85da-b42e99c9b482, 0205796..."
1,1,"[0746b54f-53c4-11ee-85da-b42e99c9b482, 01bb239..."
2,2,"[02097220-b0f1-11ec-85d9-b42e99c9b482, 08084ff..."
3,3,"[0d43ef1f-cc93-11ed-85da-b42e99c9b482, 0d43ef1..."
4,4,"[17f5a3ac-d3fd-11eb-80cc-901b0e95a2a8, 17f5a3a..."
...,...,...
213,382,"[03238f50-7091-11ed-85da-b42e99c9b482, 0a6e05e..."
214,383,"[03238f66-7091-11ed-85da-b42e99c9b482, 03238f6..."
215,389,"[0205796c-ae0d-11ed-85da-b42e99c9b482, 0205796..."
216,393,"[03238f5a-7091-11ed-85da-b42e99c9b482, 03238f5..."


In [5]:
# Создание словаря с соответствием ID товара и списком товаров в одном кластере
product_clusters = {}
for _, row in cluster_groups.iterrows():
    cluster_id = row['nn_clusters_relu']
    product_ids = row['uid']
    for product_id in product_ids:
        if product_id not in product_clusters:
            product_clusters[product_id] = []
        for pid in product_ids:
            if pid != product_id and pid not in product_clusters[product_id]:
                product_clusters[product_id].append(pid)


In [6]:
import chardet
file = 'разметка без стат.csv'
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'windows-1251',
 'confidence': 0.9225080522655392,
 'language': 'Russian'}

In [12]:
df_markup = pd.read_csv('разметка без стат.csv', encoding='cp1251', sep=',', low_memory=False)
pd.set_option('display.max_columns', None)
df_markup.drop('Unnamed: 0', axis= 1 , inplace= True )
df_markup.head(5)

,target_cat2_name,add1,add2,add3,add4,add5,add6,add7,add8
0,Туристические газовые горелки,Газовые баллоны,Чайники и кофеварки,Наборы туристической посуды,Сковороды,Аксессуары к горелкам и лампам,Туристические палатки,Туристические рюкзаки,Туристические спальные мешки
1,Газовые баллоны,Туристические газовые горелки,Наборы туристической посуды,Туристические палатки,Туристические рюкзаки,Туристические спальные мешки,Миски,Чашки,Вилки и ложки
2,Аксессуары к горелкам и лампам,Туристические газовые горелки,Наборы туристической посуды,Туристические фонари,Туристические рюкзаки,Кухонные принадлежности,Миски,Чашки,Вилки и ложки
3,Жидкотопливные и мультитопливные горелки,Наборы туристической посуды,Чайники и кофеварки,Сковороды,Кухонные принадлежности,Аксессуары к горелкам и лампам,Экстремальные спальные мешки,Экстремальные палатки,Утепленная одежда
4,Газовые плиты,Газовые баллоны,Наборы туристической посуды,Сковороды,Кухонные принадлежности,Кемпинговые палатки,Столы для кемпинга,Стулья и кресла для кемпинга,Туристические фонари


In [13]:
# Создаем словарь по разметке
murkup_dict = {}
for index, row in df_markup.iterrows():
    key = row['target_cat2_name']
    values = row.drop('target_cat2_name').tolist()
    murkup_dict[key] = values

In [15]:
# murkup_dict

In [17]:
# Создаем список целевых категорий, с вхождением в которые потом будем сравнивать ключи нашего словаря кластеров
target_cat = df_markup['target_cat2_name'].tolist()
target_cat

['Туристические газовые горелки',
 'Газовые баллоны',
 'Аксессуары к горелкам и лампам',
 'Жидкотопливные и мультитопливные горелки',
 'Газовые плиты',
 'Газовые и бензиновые лампы',
 'Наборы мебели',
 'Шезлонги и кровати',
 'Шкафы',
 'Столы для кемпинга',
 'Стулья и кресла для кемпинга',
 'Туристические палатки',
 'Бивуачные мешки',
 'Гамаки',
 'Экстремальные палатки',
 'Кемпинговые палатки',
 'Тенты и противомоскитные пологи',
 'Аксессуары к палаткам',
 'Рюкзаки для бега и мультиспорта',
 'Спортивные рюкзаки',
 'Велорюкзаки',
 'Городские рюкзаки',
 'Туристические рюкзаки',
 'Альпинистские рюкзаки',
 'Горнолыжные рюкзаки',
 'Рюкзаки для переноски детей',
 'Герморюкзаки',
 'Фоторюкзаки',
 'Аксессуары для рюкзаков',
 'Детские рюкзаки',
 'Одеяла',
 'Аксессуары к спальным мешкам',
 'Экстремальные спальные мешки',
 'Подушки туристические',
 'Туристические спальные мешки',
 'Кемпинговые спальные мешки',
 'Вкладыши в спальные мешки',
 'Органайзеры для багажа',
 'Аптечки',
 'Гермоснаряжение',

In [ ]:
#target_cat список целевых категорий
#murkup_dict словарь разметки
#product_clusters словарь uid : список сопутствующих uid
#df 

In [22]:
# Фильтрация товаров по категориям
result = {}
filtered_product_clusters = {}
for cluster, products in product_clusters.items():
    cluster_cat2 = df.loc[df['uid'] == cluster, 'category2_name'].iloc[0]
    # Проверяем, что cluster_cat2 не равно None
    if cluster_cat2 is not None:
        # Проверяем, что ключ словаря - товар из целевой категории
        if cluster_cat2 in target_cat:
            filtered_products = []
            for product in products:
                # Проверяем, что значения для ключа - товары из размеченных категорий 
                # Получаем список названий категорий элемента из значений
                prod_categories = df.loc[df['uid'] == product, 'category1_name':'category4_name'].values.flatten().tolist()
                # Убиваем все нулевые уровни
                prod_categories = list(filter(None, prod_categories))
                # Сравниваем их с размеченными 
                for category in prod_categories:
                    # Перебираем все оставшиеся ненулевые уровни иерархии 
                    if category in murkup_dict.get(cluster_cat2):        
                        filtered_products.append(product)
            result[cluster] = filtered_products
    else:
        print(f"Для кластера {cluster} не найдена категория")


#Кажется, то что дальше нам не надо вообще. и надо брать result...
# И не понятно, как это работало, почему апдейт вообще проходил, на каком моменте filtered_product_clusters записано??


# Теперь обновляем product_clusters только с отфильтрованными товарами 
#product_clusters.update(filtered_product_clusters)

In [23]:
# Преобразование в JSON
import json
json_data = json.dumps(product_clusters, indent=4)

# Запись JSON в файл
with open('product_clusters.json', 'w') as json_file:
    json_file.write(json_data)


In [ ]:
# проверка на вшивость всего файла. Делаем такой же json, толбко с названиями товаров и посмотрим внутрь.

In [ ]:
#target_cat список целевых категорий
#murkup_dict словарь разметки
#product_clusters словарь uid : список сопутствующих uid
#df uid-название-иерархия-кластеры

In [25]:
# Создаем пустой словарь для нового соответствия на основе названий товаров
prod_clusters_names = {}

# Проходим по каждой паре ключ-значение в словаре dict1
for key, values in product_clusters.items():
    # Находим соответствующее название товара по ключу в DataFrame
    product_name = df.loc[df['uid'] == key, 'name'].iloc[0]
    
    # Находим названия товаров для значений
    values_names = [df.loc[df['uid'] == value, 'name'].iloc[0] for value in values]
    
    # Добавляем соответствующую пару ключ-значение в новый словарь prod_clusters_names = {}

    prod_clusters_names[product_name] = values_names
#4053 sec  !! ужас как долго работает, надо делать через словарь может быть??? А не через обращения к датафрейму

In [28]:
import json
json_data = json.dumps(prod_clusters_names, ensure_ascii = False, indent=4)

# Запись JSON в файл
with open('product_clusters_names.json', 'w') as json_file:
    json_file.write(json_data)
